# Load polygon values

This is a demo that queries a polygon and evaluates near- and long-term risks to extreme heat based on the future [Extreme Heat](https://data.spatiafi.com/dataset/13) datasets.  We start by loading our App Credentials, and then using the [geojson statistics](https://docs.spatiafi.com/api/#operation/geojson_statistics_api_statistics_post) endpoint to fetch extreme heat risk values at the points of interest.

---

Install `spatiafi` (if not already installed):

In [ ]:
!sudo apt -y install libgeos-dev

In [ ]:
%pip install --upgrade spatiafi cartopy xarray

In [ ]:
import numpy as np
import spatiafi

In [ ]:
session = spatiafi.get_session()

In [ ]:
session

## Polygon

In [ ]:
url = "https://api.spatiafi.com/api/statistics"

params = {
    "item_id": "ce-extreme-heat-risk-projection-global-v1.0-ssp585-2020",
    "bidx": "1",
}

# North Saudi Arabia
payload = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [31.76491069149293, 32.29986592075784],
                [31.76491069149293, 23.092931581000542],
                [51.58140483659537, 23.092931581000542],
                [51.58140483659537, 32.29986592075784],
                [31.76491069149293, 32.29986592075784],
            ]
        ],
        "type": "Polygon",
    },
}

# `POST` the request using our `session` object, which will automatically handle authentication.
response = session.post(url, json=payload, params=params)

data = response.json()
print(data)

## Polygon Timeseries

In [ ]:
url = "https://api.spatiafi.com/api/timeseries/statistics"

params = {
    "coll_id": "spatiafi-extreme-heat-risk-projections-global-v1.0-ssp585",
    "datetime": "2020/2100",
    "bidx": "1",
}

# North Saudi Arabia
payload = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "coordinates": [
                    [
                        [31.76491069149293, 32.29986592075784],
                        [31.76491069149293, 23.092931581000542],
                        [51.58140483659537, 23.092931581000542],
                        [51.58140483659537, 32.29986592075784],
                        [31.76491069149293, 32.29986592075784],
                    ]
                ],
                "type": "Polygon",
            },
            "properties": {},
            "id": 0,
        }
    ],
}

# `POST` the request using our `session` object, which will automatically handle authentication.
response = session.post(url, json=payload, params=params)
response.raise_for_status()

data = response.json()
print(data)

In [ ]:
# Access the features dictionary
features = data["features"]

# Extract the mean values for each decade
decades = []
means = []

for feature in features:
    properties = feature["properties"]
    statistics = properties["statistics"]

    for date, stats in statistics.items():
        decade = int(date[:4])
        mean = stats["mean"]

        decades.append(decade)
        means.append(mean)

# Convert lists to NumPy array
decades = np.array(decades)
means = np.array(means)

# Print the arrays
print("Decades:", np.flipud(decades))
print("Means:", np.flipud(means))

So the extreme heat index in this region of Saudi Arabia is expected to more than triple by 2090 under a pessimistic emissions scenario.